# Setup Environment

In [ ]:
from fastai.tabular.all import *

In [ ]:
path = Path('../input/tabular-playground-series-mar-2021')

# Import Train Data

In [ ]:
train_df = pd.read_csv(path/'train.csv')
train_df['target'] = train_df['target'].astype(np.float32)
train_df

In [ ]:
train_df.describe()

In [ ]:
test_df = pd.read_csv(path/'test.csv')
test_df

In [ ]:
test_df.describe()

# Create Dataloaders

In [ ]:
# get category data column
cat_nn = list(train_df.iloc[:,1:20])

In [ ]:
# get continuous data column
cont_nn = list(train_df.iloc[:,21:-1])

In [ ]:
# split last 20% for validation
splits = (list(range(240000)),list(range(240000,300000)))

In [ ]:
procs_nn = [Categorify, FillMissing, Normalize]
to_nn = TabularPandas(train_df, procs_nn, cat_nn, cont_nn,
                      splits=splits, y_names='target')

In [ ]:
# create dataloaders
dls = to_nn.dataloaders(1024)

# Create learner

In [ ]:
# create a learner using tabular_learner
learn = tabular_learner(dls, y_range=(0,1), layers=[500,250],
                        n_out=1, loss_func=F.mse_loss)

In [ ]:
# to find a good lr
learn.lr_find()

In [ ]:
learn.fit_one_cycle(4, 1e-3)

# Get prediction

In [ ]:
tst_dl = learn.dls.test_dl(test_df)
pred, _ = learn.get_preds(dl = tst_dl)

# Make Submission File

In [ ]:
submission_df = pd.read_csv(path/'sample_submission.csv')
submission_df['target'] = pred.squeeze()
submission_df.to_csv('submission.csv', index=False)
submission_df